# Introduction to Iris

* from https://scitools.org.uk/iris/docs/latest/ : "Iris implements a data model based on the CF conventions giving you a powerful, format-agnostic interface for working with your data. It excels when working with multi-dimensional Earth Science data, where tabular representations become unwieldy and inefficient. "
* **Iris** is useful for using with Netcdf files, if they are written with CF conventions in mind. 
* The MetOffice uses Iris with their fieldsfiles (.pp)


First let's do some importing

In [1]:
import iris # import the Iris module

In [23]:
import matplotlib.pyplot as plt # matplotlib is going to be useful later
import numpy as np

To start we will explore the principal data structure used in Iris: The Cube. We will explore how to call, write, and replace attributes, data, and coordinates contained within a Cube.

## Loading cubes

There are different ways to load cubes: load(), load_cube() are the two main functions I use, and each work a little differently:

In [8]:
path_to_data = '../data/4p4km_air_potential_temperature_1.nc' # just the path to where we keep our data file plus the file name
cubes = iris.load(path_to_data)

In [9]:
print(cubes)

0: air_potential_temperature / (K)     (time: 12; model_level_number: 71; grid_latitude: 340; grid_longitude: 340)
1: surface_altitude / (m)              (grid_latitude: 340; grid_longitude: 340)


In [10]:
print(cubes[0])

air_potential_temperature                    (time: 12; model_level_number: 71; grid_latitude: 340; grid_longitude: 340)
     Dimension coordinates:
          time                                    x                       -                  -                    -
          model_level_number                      -                       x                  -                    -
          grid_latitude                           -                       -                  x                    -
          grid_longitude                          -                       -                  -                    x
     Auxiliary coordinates:
          forecast_period                         x                       -                  -                    -
          atmosphere_hybrid_height_coordinate     -                       x                  -                    -
          sigma                                   -                       x                  -                    -
          s

In [14]:
cube = iris.load_cube(path_to_data, 'air_potential_temperature')
cube

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


You can also use iris.load() and load_cube() with a contraints object to single out specific cubes:

In [26]:
name_constraint = iris.AttributeConstraint(STASH = 'm01s00i004')
cube = iris.load(path_to_data, constraint)
cube

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


Notice how load() will always return a list of cubes, even if only a single one is specified.

In [27]:
type(cube)

iris.cube.CubeList

In Iris, a list of cubes comes as it's own special object: a CubeList

In [28]:
cube = iris.load_cube(path_to_data, constraint)
cube

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


Not only can you constrain which cubes you want to load, but which parts of it, e.g. a specific model level:

In [36]:
level_constraint = iris.Constraint(model_level_number = np.arange(0,10))

cube10 = iris.load(path_to_data, name_constraint & level_constraint)[0]
cube10

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,10,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


You can also load data from multiple files at once; this is useful as often you keep terrain data in a seperate file, but need it to recalculate the height coordinates

In [40]:
paths = ['../data/4p4km_air_potential_temperature_1.nc', '../data/4p4km_upward_air_velocity_1.nc']
cubes_from_two = iris.load(paths)
cubes_from_two

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


In [48]:
cubes = iris.load(paths, ['air_potential_temperature', 'upward_air_velocity'])
cubes

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-


## Exploring the cube

Now we will briefly go through the various ways of exploiting the cube, starting with calling and re-writing attributes and metadata, and extracting data and coordinates